# websearch agent

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pocketflow import Node,Flow
import yaml
load_dotenv()

True

In [3]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [5]:
# 决定节点：作用是判断问题是直接返回回答还是继续search

class DecideAction(Node):
    def prep(self, shared):
        context = shared.get("context","没有之前的搜索")
        question = shared['question']

        return question, context
    
    def exec(self, prep_res):
        question, context = prep_res
        print("Agent 正在决定下一步步骤...")
        prompt = f"""
            ### 背景信息
            你是一名可以进行网络搜索的研究助手。
            问题：{question}
            以往研究：{context}
            ### 行动范围
            [1] search
            描述：在网络上查找更多信息
            参数：query（字符串）：要搜索的内容

            [2] answer
            描述：用现有知识回答问题
            参数：answer（字符串）：问题的最终答案
            
            ### 下一步行动
            根据背景信息和可用行动，决定下一步行动。
            请按以下格式返回你的回应：

            ```yaml
            thinking: |
                <你的分步推理过程>
            action: search 或 answer
            reason: <你选择该行动的原因>
            answer: <如果行动是answer，填写此处>
            search_query: <如果行动是search，填写具体的搜索查询>
            ```
            
            ###重要提示：请务必做到：
            所有多行字段使用适当的缩进（4 个空格）
            多行文本字段使用 | 字符
            单行字段不使用 | 字符
            """

        res = call_llm(prompt)
        yaml_str = res.split("```yaml")[1].split("```")[0].strip()

        return yaml.safe_load(yaml_str)

    def post(self, shared, prep_res, exec_res):
        if exec_res['action'] == 'search':
            shared['search_query'] = exec_res['search_query']
            print(f"Agent 决定去搜索：{shared['search_query']}")
        else:
            shared['context'] = exec_res['answer']
            print("Agent 决定回答答案")

        return exec_res['action']

In [6]:
# 进行知识搜索--应该通过搜索引擎，这里直接返回进行演示
def search_engine(query: str):
    res = """
        LlamaFirewall是Meta开发的LLM安全防护框架，其中包括了四个组件构成四道防线，但是在实际使用中可根据需要使用其中一个或多个组件：
        1）	PromptGuard 是框架的第一道防线，基于轻量级 BERT 分类器专注于快速识别直接的提示注入攻击。它以超低延迟处理用户输入和不可信数据，特别擅长捕捉经典的越狱模式和社会工程学攻击，为高吞吐量环境提供了理想的安全保障
        2）	AlignmentCheck 作为框架的深度监控层，通过实时审计 LLM 代理的推理过程，利用少样本提示和语义分析技术检测目标劫持和间接提示注入。它能够确保 AI 系统的决策始终与用户意图保持一致，即使是面对不透明或黑盒模型也能有效工作。
        3）	Regex + Custom Scanners 提供了灵活的自定义安全规则层，通过配置正则表达式和简单 LLM 提示来识别已知攻击模式和不良行为。这个组件支持跨语言检测，使组织能够快速应对新出现的威胁。
        4）	CodeShield 作为专用的代码安全守护者，对 LLM 生成的代码进行实时静态分析，支持 Semgrep 和正则规则，覆盖八种编程语言。它能有效防止不安全代码被执行或部署，为代码生成应用提供了必要的安全保障。
        """
    return res

In [7]:
# search节点：作用是搜索答案
class SearchWeb(Node):
    def prep(self, shared):
        return shared['search_query']
    

    def exec(self, prep_res):
        print(f"Agent 正在从网络搜索：{prep_res}")
        res = search_engine(prep_res)

        return res
    

    def post(self, shared, prep_res, exec_res):
        previous = shared.get("context", "")
        shared["context"] = previous + "\n\nSEARCH:" + shared['search_query'] + "\nRESULT:" + exec_res
        print("Agent 搜索到数据，正在分析...")
        
        return 'decide'


In [8]:
# 回答问题节点
class AnswerQuestion(Node):
    def prep(self, shared):
        return shared['question'], shared.get("context", "")
    
    def exec(self, prep_res):
        question, context = prep_res
        print("Agent 正在回答问题...")
        prompt = f"""
            ## 背景信息
            根据以下信息，回答问题。
            问题：{question}
            研究资料：{context}
            
            ## 你的回答：
            利用研究结果提供全面的答案。
            """
        
        answer = call_llm(prompt=prompt)
        return answer

    
    def post(self, shared, prep_res, exec_res):
        shared['answer'] = exec_res
        
        return 'done'

In [9]:
def create_travel_agent_flow():
    decide_node = DecideAction()
    search_node = SearchWeb()
    answer_node = AnswerQuestion()

    # 表示当 decide_node 节点的决策结果为 "search" 时，工作流将跳转到 search_node 节点继续执行
    decide_node  - "search" >> search_node  

    # 表示当 decide_node 节点的决策结果为 "answer" 时，工作流将跳转到 answer_node 节点继续执行
    decide_node  - "answer" >> answer_node
    
    # 表示当 search_node 节点执行完毕后，返回 "decide" 信号，工作流将回到 decide_node 节点重新进行决策
    search_node  - "decide" >> decide_node

    return Flow(start=decide_node)

In [13]:
def main():
    shared = {
        "question": "LlamaFirewall有哪些组件？"
    }
    travel_agent_flow = create_travel_agent_flow()
    travel_agent_flow.run(shared)
    print (shared.get("answer", "俺不知道耶～😅"))

if __name__ == '__main__':
    main()

Agent 正在决定下一步步骤...
Agent 决定去搜索：LlamaFirewall components
Agent 正在从网络搜索：LlamaFirewall components
Agent 搜索到数据，正在分析...
Agent 正在决定下一步步骤...
Agent 决定回答答案
Agent 正在回答问题...
LlamaFirewall 是由 Meta 开发的针对大语言模型（LLM）的安全防护框架，其包含以下四个主要组件：

1. **PromptGuard**：用于快速识别提示注入攻击，保护模型免受恶意输入的影响。  
2. **AlignmentCheck**：用于实时审计模型的推理过程，确保其行为符合预期的安全和伦理标准。  
3. **Regex + Custom Scanners**：提供灵活的自定义安全规则，允许用户根据特定需求配置安全策略。  
4. **CodeShield**：用于对模型生成的代码进行实时安全分析，防止潜在的代码级威胁。

这些组件既可以单独使用，也可以组合集成，以应对不同应用场景下的安全威胁。
